## Setup LangTrace and Weaviate

* Sign-up for a free account on [LangTrace](https://langtrace.ai/.com "Title").

* Create a Project and generate an API Key

* <span style="color:red"> Note you will also sign up and get an API key from [OPENAI](https://OPENAI.COM/ "OPEN AI") if you havent done so already </span>


* Setup a cluster on [Weaviate](https://www.weaviate.io/ "Weaviate") and retrieve the cluster API key and cluster url

## Install packages and setup environment

In [ ]:
pip install -U langtrace-python-sdk weaviate-client openai

In [ ]:

import os
os.environ['LANGTRACE_API_KEY'] = "YOUR_LANGTRACE_API_KEY"
os.environ['WCD_API_KEY'] = "YOUR_CLUSTER_API_KEY" #cluster api key
os.environ['WCD_URL'] = "YOUR_CLUSTER_URL" #cluster url key
os.environ['OPENAI_APIKEY'] = "YOUR_OPEN_AI_KEY"

<br>

## Imports

In [ ]:
from langtrace_python_sdk import langtrace # Must precede any llm module imports
langtrace.init(api_key = os.environ['LANGTRACE_API_KEY'])

import weaviate
import weaviate.classes as wvc
import requests
import json

In [ ]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WCD_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCD_API_KEY")),
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]  # Replace with your inference API key
    }
)

<br>

## Create an collection, insert some data and make a query using Weaviate

In [ ]:

# ===== define collection =====
questions = client.collections.create(
    name="Question1",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    generative_config=wvc.config.Configure.Generative.openai()  # Ensure the `generative-openai` module is used for generative queries
)

In [ ]:
# ===== insert some data =====
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

question_objs = list()
for i, d in enumerate(data):
    question_objs.append({
        "answer": d["Answer"],
        "question": d["Question"],
        "category": d["Category"],
    })

questions = client.collections.get("Question")
questions.data.insert_many(question_objs)

In [ ]:
# ===== run a query =====
response = questions.query.near_text(
        query="biology",
        limit=2
    )
print(response)

<br>

## You can now view your traces on the Langtrace dashboard

![image info](./assets/image1.png)